In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import product

In [2]:
class Dummy_obj:
    def __init__(self, onnx, vnnlib, n_splits, ot, st, bt, res_o, res_s, res_b):
        self.onnx, self.vnnlib, self.n_splits = onnx, vnnlib, int(n_splits)
        self.ot, self.st, self.bt = float(ot), float(st), float(bt)
        self.res_o, self.res_s, self.res_b = res_o, res_s, res_b
        self.skip = True if n_splits==-1 else False
        
    def __str__(self):
        return f"Instance: {self.onnx}, {self.vnnlib}, {self.n_splits}, {self.res_o}, {self.res_b}, {self.res_s}, {self.ot}, {self.bt}, {self.st}"

In [3]:
def load_csv(file):
    with open(file) as f:
        f.readline()
        rows = []
        for l in f:
#             onnx,vnnlib,strategy,mask,n_splits,original_r,original_t,splitted_r,splitted_t,baseline_r,baseline_t
            fileds = l.split(',')
            assert len(fileds)==11
            if fileds[4] == "SKIPPED":
                rows.append( Dummy_obj(fileds[0], fileds[1], -1, -1, -1, -1, -1, -1, -1) )
                continue
            if fileds[4] == "SPLIT_FAILED":
                rows.append( Dummy_obj(fileds[0], fileds[1], -1, -1, -1, -1, -1, -1, -1) )
                continue
            rows.append( Dummy_obj(fileds[0], fileds[1], fileds[4], 
                                   fileds[6], fileds[8], fileds[10], 
                                   fileds[5], fileds[7], fileds[9]) )
    return (file.stem, rows)
            

In [4]:
exp2_path = Path("/Users/linhan/relusplitter/experiment/results/exp2")

def get_ress(benchmark_str):
    res_abcrown = []
    res_nsat    = []
    res_nnenum  = []

    for f in exp2_path.iterdir():
        if f.suffix != ".csv":
            continue
        if "marabou" in f.stem:
            continue
        if benchmark_str not in f.stem:
            continue

        if "abcrown" in f.stem:
            res_abcrown.append(f)
        elif "neuralsat" in f.stem:
            res_nsat.append(f)
        elif "nnenum" in f.stem:
            res_nnenum.append(f)

    #     results.append(load_csv(f))
    #     print(f.stem)
    func = lambda x: x.stem.split("~")[-2]+x.stem.split("~")[-3]
    res_abcrown.sort(key=func)
    res_nsat.sort(key=func)
    res_nnenum.sort(key=func)
    print(len(res_abcrown))
    print(len(res_nsat))
    print(len(res_nnenum))
    return res_abcrown, res_nsat, res_nnenum
    


In [5]:
from math import log



# not really slowdown but just diff in veri time
def cal_slowdown(dp):
    if dp.st == -1:
        return 0
    if dp.res_s == "timeout":
#         print("TO overflow")
        pass
    return dp.st - dp.ot


# same metric, but div by num of neurons
def cal_slowdown_per_neuron(dp):
    if dp.st == -1:
        return 0
    return cal_slowdown(dp) / dp.n_splits

In [6]:
decimals = 3
def format_number(num):
    return f"{num:.{decimals}f}"

# ACAS XU

In [7]:
res_abcrown, res_nsat, res_nnenum = get_ress("acas")

5
5
5


In [8]:

fnames = []
rows = []

for r in res_abcrown:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< abcrown >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< abcrown >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
0.093 & 0.008 & 0.036 & 0.019 & 0.031
0.616 & 0.136 & 0.608 & 0.123 & 0.631


In [9]:

fnames = []
rows = []

for r in res_nsat:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< neuralsat >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< neuralsat >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
0.293 & 0.046 & 0.094 & 0.105 & 0.016
2.943 & 0.667 & 1.077 & 1.090 & 0.201


In [10]:

fnames = []
rows = []

for r in res_nnenum:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< nnenum >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< nnenum >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
0.183 & 0.262 & 0.031 & 0.139 & 0.151
1.940 & 4.941 & 0.530 & 3.963 & 2.722


# MNIST

In [11]:
res_abcrown, res_nsat, res_nnenum = get_ress("mnist")

5
5
5


In [12]:

fnames = []
rows = []

for r in res_abcrown:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< abcrown >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< abcrown >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
5.117 & 0.003 & 1.863 & 1.334 & 0.450
73.367 & 0.722 & 54.851 & 42.984 & 13.327


In [13]:

fnames = []
rows = []

for r in res_nsat:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< neuralsat >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< neuralsat >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
1.527 & 0.135 & 0.717 & 0.608 & 0.358
16.883 & 27.893 & 24.223 & 21.839 & 12.333


In [14]:

fnames = []
rows = []

for r in res_nnenum:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< nnenum >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< nnenum >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
6.553 & 0.499 & 3.362 & 1.886 & 4.039
75.443 & 113.195 & 78.474 & 51.338 & 87.205


# TLL

In [15]:
res_abcrown, res_nsat, res_nnenum = get_ress("tll")

5
5
0


In [16]:

fnames = []
rows = []

for r in res_abcrown:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< abcrown >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< abcrown >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
0.105 & 0.006 & 0.003 & 0.003 & 0.003
7.562 & 9.535 & 12.820 & 12.059 & 12.096


In [17]:

fnames = []
rows = []

for r in res_nsat:
    fname, dps = load_csv(r)
    fnames.append(fname)
    rows.append(dps)

cnt = 0
spn_total = [0,0,0,0,0]
slowdown_total  = [0,0,0,0,0]
for d1,d2,d3,d4,d5 in zip(*rows):
    assert d1.onnx == d2.onnx == d3.onnx == d4.onnx == d5.onnx
    assert d1.vnnlib == d2.vnnlib == d3.vnnlib == d4.vnnlib == d5.vnnlib
    dps = [d1, d2, d3, d4, d5]
    if any([i.skip for i in dps]):
        continue
        
    spns = [cal_slowdown_per_neuron(dp) for dp in dps]
    slowdowns = [cal_slowdown(dp) for dp in dps]
    spn_total = [a+b for a,b in zip(spn_total, spns)]
    slowdown_total = [a+b for a,b in zip(slowdown_total, slowdowns)]
    cnt += 1
        
    
print("<< neuralsat >>")
print([i.split("~")[2:-1] for i in fnames])
formatted_spn = [format_number(i/cnt) for i in spn_total]
print(" & ".join(formatted_spn))

# Format and print slowdown_total
formatted_slowdown = [format_number(i/cnt) for i in slowdown_total]
print(" & ".join(formatted_slowdown))

<< neuralsat >>
[['stable+', 'random'], ['stable-', 'random'], ['unstable', 'random'], ['unstable', 'reluS+'], ['unstable', 'reluS-']]
0.078 & 0.005 & 0.003 & 0.002 & 0.002
5.558 & 6.960 & 9.562 & 8.949 & 8.936
